In [ ]:
import numpy as np
import pandas as pd
import warnings
import math
import copy
from ..config.parameters import *

class SystemModel(object):
    def __init__(self):
        self.UAVspeed = UAV_Speed
        self.User_number = NumberOfUsers
        self.UserperCell = UserNumberPerCell
        
        # Initialize UAVs and positions
        self.U_idx = np.arange(NumberOfUAVs)
        self.PositionOfUAVs = pd.DataFrame(
            np.zeros((3, NumberOfUAVs)),
            columns=self.U_idx.tolist(),
        )
        self.PositionOfUAVs.iloc[0, :] = [100, 200, 400]  # UAV initial x
        self.PositionOfUAVs.iloc[1, :] = [100, 400, 400]  # UAV initial y
        self.PositionOfUAVs.iloc[2, :] = [100, 100, 100]  # UAV initial height

        self.Zone_border_X = ServiceZone_X
        self.Zone_border_Y = ServiceZone_Y
        self.Zone_border_Z = Hight_limit_Z

        # Initialize users and positions
        self.K_idx = np.arange(NumberOfUsers)
        self.PositionOfUsers = pd.DataFrame(
            np.random.random((3, NumberOfUsers)),
            columns=self.K_idx.tolist(),
        )
        self.PositionOfUsers.iloc[0,:] = [162, 176, 227, 383, 458, 43]
        self.PositionOfUsers.iloc[1, :] = [117, 62, 183, 426, 397, 328]
        self.PositionOfUsers.iloc[2, :] = 0  # users have no height

        # Store initial positions
        self.Init_PositionOfUsers = copy.deepcopy(self.PositionOfUsers)
        self.Init_PositionOfUAVs = copy.deepcopy(self.PositionOfUAVs)
        
        # State for neural network input
        self.State = np.zeros([1, NumberOfUAVs * 3 + NumberOfUsers], dtype=float)

        # Power allocation
        self.Power_allocation_list = pd.DataFrame(
            np.ones((1, NumberOfUsers)),
            columns=np.arange(NumberOfUsers).tolist(),
        )
        self.Power_unit = UAV_power_unit
        self.Power_allocation_list = self.Power_allocation_list * self.Power_unit

    def User_randomMove(self, MAXspeed, NumberofUsers):
        self.PositionOfUsers.iloc[[0,1],:] += np.random.randn(2, NumberofUsers) * MAXspeed

    def User_Purposive_Move_6(self, MAXspeed):
        self.PositionOfUsers.iloc[0, [1, 2]] += MAXspeed
        self.PositionOfUsers.iloc[1, [0]] += MAXspeed

    def Get_Distance_U2K(self, UAV_Position, User_Position, UAVsnumber, Usersnumber):
        Distence = pd.DataFrame(
            np.zeros((UAVsnumber, Usersnumber)),
            columns=np.arange(Usersnumber).tolist(),
        )
        
        for i in range(UAVsnumber):
            for j in range(Usersnumber):
                Distence.iloc[i,j] = np.linalg.norm(UAV_Position.iloc[:,i] - User_Position.iloc[:,j])
                
        return Distence

    def Get_Propergation_Loss(self, distence_U2K, UAV_Position, UAVsnumber, Usersnumber, f_c):
        Propergation_Loss = pd.DataFrame(
            np.zeros((UAVsnumber, Usersnumber)),
            columns=np.arange(Usersnumber).tolist(),
        )

        for i in range(UAVsnumber):
            for j in range(Usersnumber):
                UAV_Hight = UAV_Position.iloc[2,i]
                D_H = np.sqrt(np.square(distence_U2K.iloc[i,j]) - np.square(UAV_Hight))
                d_0 = np.max([(294.05 * math.log(UAV_Hight,10) - 432.94), 18])
                p_1 = 233.98 * math.log(UAV_Hight,10) - 0.95
                
                if D_H <= d_0:
                    P_Los = 1.0
                else:
                    P_Los = d_0/D_H + math.exp(-(D_H/p_1) * (1-(d_0/D_H)))

                if P_Los > 1:
                    P_Los = 1

                P_NLos = 1 - P_Los

                L_Los = 30.9 + (22.25-0.5*math.log(UAV_Hight,10))*math.log(distence_U2K.iloc[i,j],10) + 20*math.log(f_c,10)
                L_NLos = np.max([L_Los, 32.4+(43.2-7.6*math.log(UAV_Hight,10))*math.log(distence_U2K.iloc[i,j],10)+20*math.log(f_c,10)])
                Avg_Los = P_Los*L_Los + P_NLos*L_NLos
                gain = np.random.rayleigh(scale=1, size=None) * pow(10,(-Avg_Los/10))
                Propergation_Loss.iloc[i,j] = gain

        return Propergation_Loss

    def Get_Channel_Gain_NOMA(self, UAVsnumber, Usersnumber, PropergationLosslist, UserAssociationlist, Noise_Power):
        ChannelGain_list = pd.DataFrame(
            np.zeros((1, Usersnumber)),
            columns=np.arange(Usersnumber).tolist(),
        )

        for j in range(Usersnumber):
            i_Server_UAV = UserAssociationlist.iloc[0, j]
            Signal_power = 10000 * PropergationLosslist.iloc[i_Server_UAV, j]
            ChannelGain = Signal_power / Noise_Power
            ChannelGain_list.iloc[0, j] = ChannelGain

        return ChannelGain_list

    def Get_SINR_NNOMA(self, UAVsnumber, Usersnumber, PropergationLosslist, UserAssociationlist, ChannelGain_list, Noise_Power):
        SINR_list = pd.DataFrame(
            np.zeros((1, Usersnumber)),
            columns=np.arange(Usersnumber).tolist(),
        )

        for j in range(Usersnumber):
            i_Server_UAV = UserAssociationlist.iloc[0,j]
            Signal_power = self.Power_allocation_list.iloc[0,j] * PropergationLosslist.iloc[i_Server_UAV,j]
            I_inter_cluster = 0

            for j_idx in range(Usersnumber):
                if UserAssociationlist.iloc[0,j_idx] == i_Server_UAV:
                    if ChannelGain_list.iloc[0,j] < ChannelGain_list.iloc[0,j_idx] and j!=j_idx:
                        I_inter_cluster = I_inter_cluster + (
                            self.Power_allocation_list.iloc[0, j_idx] * PropergationLosslist.iloc[i_Server_UAV, j]
                        )
                else:
                    Inter_UAV = UserAssociationlist.iloc[0,j_idx]
                    I_inter_cluster = I_inter_cluster + (
                        self.Power_allocation_list.iloc[0,j_idx] * PropergationLosslist.iloc[Inter_UAV,j]
                    )

            SINR = Signal_power / (I_inter_cluster + Noise_Power)
            SINR_list.iloc[0,j] = SINR

        return SINR_list
    
    def Calcullate_Datarate(self, SINRlist, Usersnumber, B):
        Daterate = pd.DataFrame(
            np.zeros((1, Usersnumber)),
            columns=np.arange(Usersnumber).tolist(),
        )
        
        for j in range(Usersnumber):
            if SINRlist.iloc[0,j] <= 0:
                print(SINRlist)
                warnings.warn('SINR wrong')
            Daterate.iloc[0,j] = B * math.log((1 + SINRlist.iloc[0,j]), 2)

        SumDataRate = sum(Daterate.iloc[0,:])
        Worst_user_rate = min(Daterate.iloc[0,:])
        return Daterate, SumDataRate, Worst_user_rate

    def Reset_position(self):
        self.PositionOfUsers = copy.deepcopy(self.Init_PositionOfUsers)
        self.PositionOfUAVs = copy.deepcopy(self.Init_PositionOfUAVs)

    def Create_state_Noposition(self, serving_UAV, User_association_list, User_Channel_Gain):
        UAV_position_copy = copy.deepcopy(self.PositionOfUAVs.values)
        UAV_position_copy[:,[0,serving_UAV]] = UAV_position_copy[:,[serving_UAV,0]]
        User_Channel_Gain_copy = copy.deepcopy(User_Channel_Gain.values[0])

        for UAV in range(NumberOfUAVs):
            self.State[0, 3 * UAV:3 * UAV + 3] = UAV_position_copy[:, UAV].T

        User_association_copy = copy.deepcopy(User_association_list.values)
        desirable_user = np.where(User_association_copy[0]==serving_UAV)[0]

        for i in range(len(desirable_user)):
            User_Channel_Gain_copy[i], User_Channel_Gain_copy[desirable_user[i]] = User_Channel_Gain_copy[desirable_user[i]], User_Channel_Gain_copy[i]

        for User in range(NumberOfUsers):
            self.State[0, (3*UAV+3)+User] = User_Channel_Gain_copy[User].T

        Stat_for_return = copy.deepcopy(self.State)
        return Stat_for_return

    def take_action_NOMA(self, action_number, acting_UAV, User_asso_list, ChannelGain_list):
        # Movement actions
        UAV_move_direction = action_number % 7
        if UAV_move_direction == 0:  # Move right
            self.PositionOfUAVs.iloc[0,acting_UAV] += self.UAVspeed
            if self.PositionOfUAVs.iloc[0,acting_UAV] > self.Zone_border_X:
                self.PositionOfUAVs.iloc[0, acting_UAV] = self.Zone_border_X
        elif UAV_move_direction == 1:  # Move left
            self.PositionOfUAVs.iloc[0, acting_UAV] -= self.UAVspeed
            if self.PositionOfUAVs.iloc[0, acting_UAV] < 0:
                self.PositionOfUAVs.iloc[0, acting_UAV] = 0
        elif UAV_move_direction == 2:  # Move forward
            self.PositionOfUAVs.iloc[1, acting_UAV] += self.UAVspeed
            if self.PositionOfUAVs.iloc[1, acting_UAV] > self.Zone_border_Y:
                self.PositionOfUAVs.iloc[1, acting_UAV] = self.Zone_border_Y
        elif UAV_move_direction == 3:  # Move backward
            self.PositionOfUAVs.iloc[1, acting_UAV] -= self.UAVspeed
            if self.PositionOfUAVs.iloc[1, acting_UAV] < 0:
                self.PositionOfUAVs.iloc[1, acting_UAV] = 0
        elif UAV_move_direction == 4:  # Move up
            self.PositionOfUAVs.iloc[2, acting_UAV] += self.UAVspeed
            if self.PositionOfUAVs.iloc[2, acting_UAV] > self.Zone_border_Z:
                self.PositionOfUAVs.iloc[2, acting_UAV] = self.Zone_border_Z
        elif UAV_move_direction == 5:  # Move down
            self.PositionOfUAVs.iloc[2, acting_UAV] -= self.UAVspeed
            if self.PositionOfUAVs.iloc[2, acting_UAV] < 20:
                self.PositionOfUAVs.iloc[2, acting_UAV] = 20
        elif UAV_move_direction == 6:  # Hover
            pass

        # Power allocation
        power_allocation_scheme = action_number // 7
        acting_user_list = np.where(User_asso_list.iloc[0,:] == acting_UAV)[0]
        cg_users_in_cluster = np.zeros(len(acting_user_list))
        for i in range(len(acting_user_list)):
            cg_users_in_cluster[i] = float(ChannelGain_list.iloc[0, acting_user_list[i]])
        
        cg_user_in_cluster_ranking = np.argsort(cg_users_in_cluster)
        
        ranked_users_in_clusters = np.zeros(len(acting_user_list))
        for i in range(len(acting_user_list)):
            ranked_users_in_clusters[i] = int(acting_user_list[np.where(cg_user_in_cluster_ranking == i)[0][0]])
        
        # Power allocation based on cluster size
        if len(acting_user_list) == 1:
            if action_number >= 63 and action_number <= 83:
                if power_allocation_scheme % 3 == 0:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = 7 * self.Power_unit
                elif power_allocation_scheme % 3 == 1:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = 3 * self.Power_unit
                elif power_allocation_scheme % 3 == 2:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = 1 * self.Power_unit

        if len(acting_user_list) == 2:
            if action_number <= 62:
                if power_allocation_scheme % 3 == 0:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit*2
                elif power_allocation_scheme % 3 == 1:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit*4
                elif power_allocation_scheme % 3 == 2:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit*7

                if power_allocation_scheme // 3 == 0:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit
                elif power_allocation_scheme // 3 == 1:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit/2
                elif power_allocation_scheme // 3 == 2:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit/4
        
        if len(acting_user_list) == 3:
            if action_number >= 84 and action_number <= 104:
                if power_allocation_scheme % 3 == 0:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[2])] = self.Power_unit * 7
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit * 4
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit * 1
                
                elif power_allocation_scheme % 3 == 1:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[2])] = self.Power_unit * 6
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit * 2
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit * 2/3
                
                elif power_allocation_scheme % 3 == 2:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[2])] = self.Power_unit * 6
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit * 3
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit * 1
        
        if len(acting_user_list) == 4:
            if action_number >= 105 and action_number <= 125:
                if power_allocation_scheme % 3 == 0:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[3])] = self.Power_unit * 7
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[2])] = self.Power_unit * 3
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit * 1
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit * 1/2
                
                elif power_allocation_scheme % 3 == 1:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[3])] = self.Power_unit * 7
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[2])] = self.Power_unit * 4
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit * 2
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit * 2/3
                
                elif power_allocation_scheme % 3 == 2:
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[3])] = self.Power_unit * 5
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[2])] = self.Power_unit * 3
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[1])] = self.Power_unit * 1
                    self.Power_allocation_list.iloc[0,int(ranked_users_in_clusters[0])] = self.Power_unit * 1/4